In [1]:
import h5py
import numpy as np
from scipy.optimize import curve_fit
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import os
import re

def fit_ImSigma_4(iw, gamma, alpha, beta, delta, delta2):
    ImSigma = -gamma - alpha * iw - beta * iw**2 - delta * iw**3 - delta2 * iw**4
    return ImSigma

def fit_ImSigma_3(iw, gamma, alpha, beta, delta):
    ImSigma = -gamma - alpha * iw - beta * iw**2 - delta * iw**3
    return ImSigma

def fit_ImSigma_2(iw, gamma, alpha, beta):
    ImSigma = -gamma - alpha * iw - beta * iw**2
    return ImSigma

def fit_ImSigma_1(iw, gamma, alpha):
    ImSigma = gamma - alpha * iw
    return ImSigma

frequencies = ["w0_2", "w0_4"]
data_np = {}
data_np["w0_4"] = {}
data_np["w0_2"] = {}
data = {}
data["w0_4"] = {}
data["w0_2"] = {}

for freq in frequencies:
    PATH = "/gpfs/data/fs72150/springerd/Projects/DynamicalU/PhaseSpace/Ust_vs_Usc/beta20/"+freq
#     print(PATH)
    U_st = []
    U_sc = []
    ivmaxs = []
    iv0s = []
    popt2s = []
    fitpoints = 10

    for subdir, dirs, files in os.walk(PATH):
        for file in files:
            if "oo_2" in file:
                U1 = re.findall(r'\d+', subdir)[4]
                U2 = re.findall(r'\d+', subdir)[5]

#                 print(U1, U2)
                if int(U1) not in U_st:
                    U_st.append(int(U1))
                if int(U2) not in U_sc:
                    U_sc.append(int(U2))

                if str(U1) not in data[freq].keys():
                    data[freq][str(U1)] = {}
                if str(U2) not in data[freq][str(U1)].keys():
                    data[freq][str(U1)][str(U2)] = {}

                filename = subdir+'/'+file
#                 print(filename)
                with h5py.File(filename, "r") as f:
                    iv = np.array(f['.axes']['iw'][:])
                    tau = np.array(f['.axes']['tau'][:])
                    siv = np.array(f['dmft-last']['ineq-001']['siw']['value'])
                    gtau = np.array(f['dmft-last']['ineq-001']['gtau']['value'])
                    ntaun0 = np.array(f['dmft-last']['ineq-001']['ntau-n0']['value'][0,0,0,1,0])
                    szsz = np.array(np.sum ( f['dmft-last']['ineq-001']['ntau-n0']['value'][0,0,0,0,:] + f['dmft-last']['ineq-001']['ntau-n0']['value'][0,0,0,1,:] )*20 )

                iv0 = int(iv.shape[0]/2)
                nn = iv0 + fitpoints
                ivmax = iv0 + 50
                iv_limit = iv[iv0:nn].real
                siv_limit = (siv[0,0,iv0:nn]).imag

    #             popt2, pcov2 = curve_fit(fit_ImSigma_1, iv_limit, siv_limit)
    #             popt2, pcov2 = curve_fit(fit_ImSigma_2, iv_limit, siv_limit)
                popt2, pcov2 = curve_fit(fit_ImSigma_4, iv_limit, siv_limit)
                popt2s.append(popt2)

    #             fit_sigma_2 = fit_ImSigma_1(iv_limit, popt2[0], popt2[1])
    #             fit_sigma_2 = fit_ImSigma_2(iv_limit, popt2[0], popt2[1], popt2[2])
                fit_sigma_2 = fit_ImSigma_4(iv_limit, popt2[0], popt2[1], popt2[2], popt2[3], popt2[4])

                data[freq][str(U1)][str(U2)]["iv"] = iv
                data[freq][str(U1)][str(U2)]["siv"] = siv
                data[freq][str(U1)][str(U2)]["gtau"] = gtau
                data[freq][str(U1)][str(U2)]["fit_siv"] = fit_sigma_2
                data[freq][str(U1)][str(U2)]["Z"] = round(1/(1+popt2[1]), 2)
                data[freq][str(U1)][str(U2)]["alpha"] = popt2[1]
                data[freq][str(U1)][str(U2)]["ntaun0"] = ntaun0
                data[freq][str(U1)][str(U2)]["szsz"] = szsz
    f = h5py.File(filename, "r")

    U_sc.sort()
    U_st.sort()

    data_np[freq]["Z"] = np.zeros((len(U_st), len(U_sc),3))
    data_np[freq]["alpha"] = np.zeros((len(U_st), len(U_sc),3))
    data_np[freq]["siv0"] = np.zeros((len(U_st), len(U_sc),3), dtype=complex)
    data_np[freq]["siv_diff"] = np.zeros((len(U_st), len(U_sc),3), dtype=complex)
    data_np[freq]["gtaubetahalf"] = np.zeros((len(U_st), len(U_sc),3))
    data_np[freq]["ntaun0"] = np.zeros((len(U_st), len(U_sc),3))
    data_np[freq]["szsz"] = np.zeros((len(U_st), len(U_sc),3))
    
#     print(U_st)
#     print(U_sc)
    k = 0
    for u in U_st:
        l = 0
        for v in U_sc:
            data_np[freq]["Z"][k,l,0] = u
            data_np[freq]["Z"][k,l,1] = v
            data_np[freq]["Z"][k,l,2] = data[freq][str(u)][str(v)]["Z"]

            data_np[freq]["alpha"][k,l,0] = u
            data_np[freq]["alpha"][k,l,1] = v
            data_np[freq]["alpha"][k,l,2] = data[freq][str(u)][str(v)]["alpha"]

            data_np[freq]["siv0"][k,l,0] = u
            data_np[freq]["siv0"][k,l,1] = v
            data_np[freq]["siv0"][k,l,2] = data[freq][str(u)][str(v)]["siv"][0,0,1000]

            data_np[freq]["siv_diff"][k,l,0] = u
            data_np[freq]["siv_diff"][k,l,1] = v
            data_np[freq]["siv_diff"][k,l,2] = data[freq][str(u)][str(v)]["siv"][0,0,1001] - data[freq][str(u)][str(v)]["siv"][0,0,1000]

            data_np[freq]["gtaubetahalf"][k,l,0] = u
            data_np[freq]["gtaubetahalf"][k,l,1] = v
            data_np[freq]["gtaubetahalf"][k,l,2] = data[freq][str(u)][str(v)]["gtau"][0,0,500]

            data_np[freq]["ntaun0"][k,l,0] = u
            data_np[freq]["ntaun0"][k,l,1] = v
            data_np[freq]["ntaun0"][k,l,2] = data[freq][str(u)][str(v)]["ntaun0"]

            data_np[freq]["szsz"][k,l,0] = u
            data_np[freq]["szsz"][k,l,1] = v
            data_np[freq]["szsz"][k,l,2] = data[freq][str(u)][str(v)]["szsz"]

            l+=1
        k+=1
    data_np[freq]["Z"][:,:,2][data_np[freq]["Z"][:,:,2]<0] = 0.0
    data_np[freq]["Z"][:,:,2][data_np[freq]["Z"][:,:,2]>1] = 0.0
    
# print(f['dmft-last']['ineq-001']['ntau-n0']['value'].shape)
# print(f['dmft-last']['ineq-001'].)
    
# print(data_np.keys())
# print(U_st)
# print(U_sc)

In [7]:
fig = make_subplots(
    rows=2, cols=3,
    specs=[[{'type': 'contour'},{'type': 'contour'},{'type': 'contour'}], [{'type': 'contour'},{'type': 'contour'},{'type': 'contour'}]],
    subplot_titles=[])

fig.add_trace(
    go.Contour(x=U_st, y=U_sc, z=data_np["w0_2"]["szsz"][:,:,2], colorscale='viridis', line_smoothing=0),
row=1, col=1)
fig.add_trace(
    go.Contour(x=U_st, y=U_sc, z=data_np["w0_2"]["ntaun0"][:,:,2], colorscale='viridis', line_smoothing=0),
row=1, col=2)
fig.add_trace(
    go.Contour(x=U_st, y=U_sc, z=data_np["w0_2"]["Z"][:,:,2], colorscale='viridis', line_smoothing=0),
row=1, col=3)

fig.add_trace(
    go.Contour(x=U_st, y=U_sc, z=data_np["w0_4"]["szsz"][:,:,2], colorscale='viridis', line_smoothing=0),
row=2, col=1)
fig.add_trace(
    go.Contour(x=U_st, y=U_sc, z=data_np["w0_4"]["ntaun0"][:,:,2], colorscale='viridis', line_smoothing=0),
row=2, col=2)
fig.add_trace(
    go.Contour(x=U_st, y=U_sc, z=data_np["w0_4"]["Z"][:,:,2], colorscale='viridis', line_smoothing=0),
row=2, col=3)


fig.update_layout(
    autosize=True,
    width=1900, height=800,
    margin=dict(l=10, r=5, b=0, t=35),
    scene1 = dict(
        zaxis_title_text ="Z",
    ),
#     scene = dict(zaxis = dict(range=[-1,1],),),
)


In [3]:
isozete = []
isoaim = 0.7
k = 0
for u in U_st:
    m = 100
    l = 0
    c = [-1,-1,-1]
    for v in U_sc:
        if np.absolute(data_np["w0_4"]["Z"][k,l,2]-isoaim) < m:
            m = np.absolute(data_np["w0_4"]["Z"][k,l,2]-isoaim)
            c = [u,v,data_np["w0_4"]["Z"][k,l,2]]
        l+=1
    isozete.append(c)
    k+=1
#     print()

print(isozete)

        

[[200, 100, 0.72], [229, 100, 0.69], [260, 80, 0.71], [290, 80, 0.68], [320, 0, 0.71], [350, 0, 0.7], [380, 20, 0.68], [409, 20, 0.66], [440, 0, 0.64], [469, 40, 0.62], [500, 140, 0.68]]


In [4]:
fig = make_subplots(
    rows=1, cols=3,
    specs=[[{'type': 'surface'},{'type': 'surface'},{'type': 'surface'}]],
#     specs=[[{'type': 'surface'},{'type': 'surface'},{'type': 'surface'}],[{'type': 'surface'},{'type': 'surface'},{'type': 'surface'}]],
    subplot_titles=[])# , subplot_titles=(titles))

fig.add_trace(
go.Surface(x=U_st, y=U_sc, z=data_np["w0_2"]["Z"][:,:,2], colorscale='viridis', showscale=False),
row=1, col=1)

fig.add_trace(
go.Surface(x=U_st, y=U_sc, z=data_np["w0_2"]["siv_diff"][:,:,2].imag, colorscale='viridis', showscale=False),
# go.Surface(x=U_st, y=U_sc, z=data_np["alpha"][:,:,2], colorscale='viridis', showscale=False),
row=1, col=2)

fig.add_trace(
go.Surface(x=U_st, y=U_sc, z=data_np["w0_2"]["gtaubetahalf"][:,:,2], colorscale='viridis', showscale=False),
row=1, col=3)


fig.update_scenes(xaxis_title_text='U_static',  
                  yaxis_title_text='U_screen')

fig.update_yaxes(title_text="Date", row = 1, col = 1)

camera = dict(
    eye=dict(x=-0.5, y=-2.5, z=1.)
)
aspectratio = dict(x=1, y=1, z=1.5)
# camera_position()


fig.update_layout(
    title=r'$\beta=20, \omega_0=4.0$',
    autosize=True,
    width=1900, height=800,
    margin=dict(l=10, r=5, b=0, t=35),
    scene1 = dict(
        zaxis_title_text ="Z",
    ),
    scene2 = dict(
        zaxis_title_text="siv(1)-siv(0)",
    ),
    scene3 = dict(
        zaxis_title_text ="gtau(beta/2)",
    ),
#     scene = dict(zaxis = dict(range=[-1,1],),),
)

# fig.write_html("/home/fs72150/springerd/Projects/DynamicalU/w4c0_beta20.html")


In [5]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'contour'},{'type': 'contour'}]],
    subplot_titles=[])

fig.add_trace(
    go.Contour(x=U_st, y=U_sc, z=data_np["w0_2"]["Z"][:,:,2], colorscale='viridis'),
row=1, col=1)
fig.add_trace(
    go.Contour(x=U_st, y=U_sc, z=data_np["w0_4"]["Z"][:,:,2], colorscale='viridis'),
row=1, col=2)